# Usage Example



## Importing the NonPredictive class from the non-predictive module:

In [25]:
import sys
import pandas as pd
sys.path.append('..')
from glsm.non_predictive.model import NonPredictive
from glsm.non_predictive.features import Feature

## Instance the model and add the features:

In [26]:
model = NonPredictive()

feature_a = Feature(
    name="Monthly Users",
    weight=0.5,
    options_df=pd.DataFrame([
            {"label": "Up to 50K", "is_ICP": False, "points": 0},
            {"label": "50K - 100K", "is_ICP": True, "points": 0},
            {"label": "100K - 200K", "is_ICP": False, "points": 0},
            {"label": "More than 200K", "is_ICP": False, "points": 0},
        ])
)
feature_b = Feature(
    name="Industry",
    weight=0.25,
    options_df=pd.DataFrame({
        'label': ['Other', 'Agriculture', 'Transportation', 'Healthcare', 'Manufacturing',
                  'Education', 'Finance', 'Technology', 'Retail', 'Telecom'],
        'is_ICP': [False, False, False, False, False, True, True, True, False, False],
        'points': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    })
)

feature_c = Feature(
    name="Mkt Investment",
    weight=1,
    options_df=pd.DataFrame({
        'label': ['Up to $50K', '50k - $100K', '100k - $200K', '$200K - $300K', '$300K - $400K', 'More than $400K'],
        'is_ICP': [False, True, False, False, False, False],
        'points': [0, 0, 0, 0, 0, 0]
    })
)

model.add_features([feature_a, feature_b, feature_c])

## Points auto assignment
You may set preview=True to check the proposed changes

In [27]:
model.auto_assign_points()

,label,is_ICP,points,Feature Name
0,Up to 50K,False,0.0,Monthly Users
1,50K - 100K,True,50.0,Monthly Users
2,100K - 200K,False,75.0,Monthly Users
3,More than 200K,False,100.0,Monthly Users
4,Other,False,0.0,Industry
5,Agriculture,False,10.0,Industry
6,Transportation,False,20.0,Industry
7,Healthcare,False,30.0,Industry
8,Manufacturing,False,40.0,Industry
9,Education,True,50.0,Industry


## Compute lambda values for a single lead:

In [28]:
lead = { 
        "Monthly Users": "50K - 100K",
        "Industry": "Technology",
        "Mkt Investment": "$300K - $400K",
    }
    
lambda_value = model.compute_lambda(lead)
lambda_value

78.57

## Read leads from a csv file and compute the lead score:

In [29]:
import pandas as pd

df = pd.read_csv('leads.csv')
df['lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)
df

,Monthly Users,Industry,Mkt Investment,lambda
0,50K - 100K,Technology,$300K - $400K,78.57
1,50K - 100K,Retail,100k - $200K,60.71
2,Up to 50K,Agriculture,50k - $100K,38.57


## Removing features

In [30]:
model.remove_features(['Mkt Investment'])

In [31]:
len(model.features)

2

### Compute new lambda after removing a feature:

In [33]:
# df with both lambdas and delta_percent
df['new_lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)

df['delta_percent'] = (df['new_lambda'] / df['lambda'] - 1)*100

df.head()


,Monthly Users,Industry,Mkt Investment,lambda,new_lambda,delta_percent
0,50K - 100K,Technology,$300K - $400K,78.57,50.0,-36.362479
1,50K - 100K,Retail,100k - $200K,60.71,55.0,-9.405370
2,Up to 50K,Agriculture,50k - $100K,38.57,2.0,-94.814623


In [34]:
round(sum( # proof of normalized weights sum to 1
    [i.normalized_weight**2 for i in model.features]),
    model.round_decimals 
)

1.0